# Comparison of model-based and MRI-based target volume definition for ocular proton therapy 
These scripts provide methods to generate polynomial tumour models, as commonly used in ocular proton therapy planning, based on 3D tumour delineations. Furthermore, it provides methods to compare the resulting model to the tumour delineations. 

More figures and details about the methods are available within the functions.

L. Klaassen, 2025. 


In [ ]:
from Prepare_base import correct_base, redefine_prom, expand_base
from Generate_tumour_model import generate_tumour_model, generate_tumour_model_extrathickness, upsample_point_cloud, save_point_cloud_as_stl, directed_angle_between_vectors
from Model_evaluation import volume_analysis, signed_surface_dist
from Automatic_measurements import Prom_Centre, LBD

import trimesh
import os
import numpy as np
import pandas as pd
import alphashape
import pymeshfix
import scipy
import shapely
import matplotlib.pyplot as plt
import sklearn
import manifold3d
from tqdm.auto import tqdm 

### Preparations

NOTE: you should specify the path to the STL files below.

In [2]:
# Define paths
filedir = r'/tmp' #Enter the path where STL files are
outdir = r'/tmp' # Enter the path where you want the models to go

tumour_path = os.path.join(filedir, 'Tumour_example.stl')
eye_path = os.path.join(filedir, 'Eye_example.stl')

if not os.path.isdir(outdir):
    os.makedirs(outdir)


In [3]:
#Load stl
tumour = trimesh.load(tumour_path)
eye = trimesh.load(eye_path)

# Tumour measurements
thickness, thickness_base, thickness_top = Prom_Centre(tumour,eye, include_sclera = False) # Prominence = thickness
lbd, lbd_coor1, lbd_coor2, base = LBD(tumour,eye)
print('Thickness is ', round(thickness,1), 'mm, largest diameter is ', round(lbd,1), 'mm')

# Define tumour base and expanded tumour base
corrected_base, corrected_base_normals = correct_base(tumour,eye, threshold_angle = 45)
expanded_base, new_points = expand_base(corrected_base, max_distance = 1.0)

# Upsample point clouds for tumour base
corrected_base, new_points2 = upsample_point_cloud(corrected_base, factor=10, k=10) # Upsample base by factor 10 with 10 nearest neighbours

# Check if thickness needs to be corrected (automatic thickness determination is less accurate in tumours with middle of the eye within or very close to tumour)
thickness_base, redefined = redefine_prom(tumour,eye, corrected_base, thickness_base)
        

Thickness is  9.1 mm, largest diameter is  17.7 mm


### Construct tumour models

In [4]:
# Construct tumour models: standard version without expansions
for sf in tqdm(np.arange(1,10,1)): #make models for shape factors 1-10 in steps of 1
    try:
        sf = round(sf,1)
        
        #location of files
        out = os.path.join(outdir, 'Testpatient_tumourmodel_sf{0}.stl'.format(sf))

        tumour_model_points = generate_tumour_model(tumour,eye, corrected_base, sf) #generate tumour model
        save_point_cloud_as_stl(tumour_model_points, out, max_attempts=5) #save tumour model as STL
        
        # Uncomment this section for a figure of the delineation and tumour model created
        #fig = plt.figure()
        #axes = fig.add_subplot(projection='3d')
        #axes.plot(eye.vertices[:,0], eye.vertices[:,1], eye.vertices[:,2], c='w', alpha = 0.8)
        #axes.scatter(tumour.vertices[:,0], tumour.vertices[:,1], tumour.vertices[:,2], c='g', alpha = 0.4, label = 'delineated tumour')
        #axes.scatter(tumour_model_points[:,0], tumour_model_points[:,1], tumour_model_points[:,2], c= 'b', alpha = 0.7, label = 'tumour model')

        #axes.axis('equal')
        #axes.set_title('Tumour model and tumour delineation SF {0}'.format(sf))
        #fig.legend()

    except Exception as e: print(e)


  0%|          | 0/9 [00:00<?, ?it/s]

   Saved STL file: /tmp/Testpatient_tumourmodel_sf1.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Saved STL file: /tmp/Testpatient_tumourmodel_sf2.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_sf3.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_sf4.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Saved STL file: /tmp/Testpatient_tumourmodel_sf5.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Mesh not a volume, upsampling points (Attempt 2)...
   Saved STL file: /tmp/Testpatient_tumourmodel_sf6.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Saved STL file: /tmp/Testpatient_tumourmodel_sf7.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_sf8.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_sf9.stl


In [5]:
# Construct tumour models: version with extra thickness
for sf in tqdm(np.arange(1,10,1)): #Make models for shape factors 1-10 in steps of 1
    try:
        sf = round(sf,1)

        #location of files
        out = os.path.join(outdir, 'Testpatient_tumourmodel_extrathickness_sf{0}.stl'.format(sf))

        tumour = trimesh.load(tumour_path)
        eye = trimesh.load(eye_path)

        tumour_model_points_extrathickness = generate_tumour_model_extrathickness(tumour,eye, corrected_base, sf, addedthickness = 0.5) #generate tumour model with 0.5 mm added thickness
        save_point_cloud_as_stl(tumour_model_points_extrathickness, out, max_attempts=5) #save tumour model as STL

        # Uncomment this section for a figure of the delineation and tumour model created
        #fig = plt.figure()
        #axes = fig.add_subplot(projection='3d')
        #axes.plot(eye.vertices[:,0], eye.vertices[:,1], eye.vertices[:,2], c='w', alpha = 0.8)
        #axes.scatter(tumour.vertices[:,0], tumour.vertices[:,1], tumour.vertices[:,2], c='g', alpha = 0.4, label = 'delineated tumour')
        #axes.scatter(tumour_model_points_extrathickness[:,0], tumour_model_points_extrathickness[:,1], tumour_model_points_extrathickness[:,2], c= 'b', alpha = 0.7, label = 'tumour model')

        #axes.axis('equal')
        #axes.set_title('Tumour model and tumour delineation SF {0}'.format(sf))
        #fig.legend()

    except Exception as e: print(e)
                

  0%|          | 0/9 [00:00<?, ?it/s]

   Mesh not a volume, upsampling points (Attempt 1)...
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf1.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf2.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf3.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf4.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf5.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Mesh not a volume, upsampling points (Attempt 2)...
   Mesh not a volume, upsampling points (Attempt 3)...
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf6.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf7.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf8.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Mesh not a volume, upsampling points (Attempt 2)...
   Saved STL file: /tmp/Testpatient_tumourmodel

In [6]:
# Construct tumour models: version with expanded base
for sf in tqdm(np.arange(1,10,1)): #make models for shape factors 1-10 in steps of 1
    try:
        sf = round(sf,1)

        #location of files
        out = os.path.join(outdir, 'Testpatient_tumourmodel_extrabase_sf{0}.stl'.format(sf))

        tumour = trimesh.load(tumour_path)
        eye = trimesh.load(eye_path)

        tumour_model_points_extrabase = generate_tumour_model(tumour,eye, expanded_base, sf) #generate tumour model 
        save_point_cloud_as_stl(tumour_model_points_extrabase, out, max_attempts=5) #save tumour model as STL

        # Uncomment this section for a figure of the delineation and tumour model created
        #fig = plt.figure()
        #axes = fig.add_subplot(projection='3d')
        #axes.plot(eye.vertices[:,0], eye.vertices[:,1], eye.vertices[:,2], c='w', alpha = 0.8)
        #axes.scatter(tumour.vertices[:,0], tumour.vertices[:,1], tumour.vertices[:,2], c='g', alpha = 0.4, label = 'delineated tumour')
        #axes.scatter(tumour_model_points_extrabase[:,0], tumour_model_points_extrabase[:,1], tumour_model_points_extrabase[:,2], c= 'b', alpha = 0.7, label = 'tumour model')

        #axes.axis('equal')
        #axes.set_title('Tumour model and tumour delineation SF {0}'.format(sf))
        #fig.legend()

    except Exception as e: print(e)


  0%|          | 0/9 [00:00<?, ?it/s]

   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf1.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf2.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf3.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf4.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf5.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf6.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf7.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf8.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf9.stl


In [7]:
# Construct tumour models: version with added thickness and expanded base
for sf in tqdm(np.arange(1,10,1)): #make models for shape factors 1-10 in steps of 1
    try:
        sf = round(sf,1)
        
        #location of files
        out = os.path.join(outdir, 'Testpatient_tumourmodel_extrathickness_extrabase_sf{0}.stl'.format(sf))
        
        tumour = trimesh.load(tumour_path)
        eye = trimesh.load(eye_path)

        tumour_model_points_extrathickness_extrabase = generate_tumour_model_extrathickness(tumour,eye, expanded_base, sf, addedthickness = 0.5) #generate tumour model with 0.5 mm added thickness
        save_point_cloud_as_stl(tumour_model_points_extrathickness_extrabase, out, max_attempts=5) #save tumour model as STL
        
        # Uncomment this section for a figure of the delineation and tumour model created
        #fig = plt.figure()
        #axes = fig.add_subplot(projection='3d')
        #axes.plot(eye.vertices[:,0], eye.vertices[:,1], eye.vertices[:,2], c='w', alpha = 0.8)
        #axes.scatter(tumour.vertices[:,0], tumour.vertices[:,1], tumour.vertices[:,2], c='g', alpha = 0.4, label = 'delineated tumour')
        #axes.scatter(tumour_model_points_extrathickness_extrabase[:,0], tumour_model_points_extrathickness_extrabase[:,1], tumour_model_points_extrathickness_extrabase[:,2], c= 'b', alpha = 0.7, label = 'tumour model')

        #axes.axis('equal')
        #axes.set_title('Tumour model and tumour delineation SF {0}'.format(sf))
        #fig.legend()

    except Exception as e: print(e)

  0%|          | 0/9 [00:00<?, ?it/s]

   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf1.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf2.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf3.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf4.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf5.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf6.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf7.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf8.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf9.stl


### Evaluate tumour models 
If tumour models have unexpected shapes, sometimes it is sufficient to run the model generation again for that shape factor.

In [8]:
# Evaluation for standard model
volume_metrics_perpatient = {}
distance_metrics_perpatient = {}

for sf in tqdm(np.arange(1,10,1)):
    try:
        sf = round(sf,1)
        tqdm.write('   started with sf {0}'.format(sf))
        model_path = os.path.join(outdir, 'Testpatient_tumourmodel_sf{0}.stl'.format(sf)) 
        model = trimesh.load(model_path)
        eye = trimesh.load(eye_path)

        volume_metrics = volume_analysis(tumour, model)
        volume_metrics_perpatient[r'SF{0}'.format(sf)] = volume_metrics

        dists, dist_metrics = signed_surface_dist(tumour, model, eye)
        distance_metrics_perpatient[r'SF{0}'.format(sf)] = dist_metrics
        #print('    ', dist_metrics)
    except Exception as e:
        print(e)
        volume_metrics_perpatient[r'SF{0}'.format(sf)] = {'overlap_abs': np.nan, 'overlap_rel': np.nan, 'overestimation_abs': np.nan,
                'overestimation_rel': np.nan, 'underestimation_abs': np.nan,
                'underestimation_rel': np.nan, 'IoU': np.nan}
        distance_metrics_perpatient[r'SF{0}'.format(sf)] = {'surf_dist_median_abs':  np.nan, 'surf_dist_min': np.nan, 'surf_dist_perc_0.5': np.nan,'surf_dist_perc_1': np.nan,'surf_dist_perc_2': np.nan,
                                                            'surf_dist_perc_5':  np.nan, 'surf_dist_perc_25':  np.nan, 
                'surf_dist_perc_50':  np.nan, 'surf_dist_perc_75':  np.nan, 'surf_dist_perc_95':  np.nan, 'surf_dist_max': np.nan}


  0%|          | 0/9 [00:00<?, ?it/s]

   started with sf 1
   started with sf 2
   started with sf 3
   started with sf 4
   started with sf 5
   started with sf 6
   started with sf 7
   started with sf 8
   started with sf 9


In [9]:
# Put the results in a dataframe
rows = []
rows2 = []

for sf, metrics in volume_metrics_perpatient.items():
    sf_number = float(sf.replace('SF', ''))
    rows.append({
            'SF': sf_number,
            'overlap_abs': metrics['overlap_abs'],
            'overlap_rel': metrics['overlap_rel'],
            'overestimation_abs': metrics['overestimation_abs'],
            'overestimation_rel': metrics['overestimation_rel'],
            'underestimation_abs': metrics['underestimation_abs'],
            'underestimation_rel': metrics['underestimation_rel'],
            'IoU': metrics['IoU'],
        })


for sf, metrics in distance_metrics_perpatient.items():
    sf_number = float(sf.replace('SF', ''))
    rows2.append({
            'SF': sf_number,
            'surf_dist_median_abs': metrics['surf_dist_median_abs'],
            'surf_dist_min': metrics['surf_dist_min'],
            'surf_dist_perc_0.5': metrics['surf_dist_perc_0.5'],
            'surf_dist_perc_1': metrics['surf_dist_perc_1'],
            'surf_dist_perc_2': metrics['surf_dist_perc_2'],
            'surf_dist_perc_5': metrics['surf_dist_perc_5'],
            'surf_dist_perc_25': metrics['surf_dist_perc_25'],
            'surf_dist_perc_50': metrics['surf_dist_perc_50'],
            'surf_dist_perc_75': metrics['surf_dist_perc_75'],
            'surf_dist_perc_95': metrics['surf_dist_perc_95']
        })

volume_df = pd.DataFrame(rows)
distance_df = pd.DataFrame(rows2)

allmetrics_df = pd.DataFrame.merge(volume_df, distance_df, how = 'left')


In [10]:
# Export dataframe
out_excel_standardmodel = os.path.join(outdir, 'Evaluation_standardmodel.xlsx')
allmetrics_df.to_excel(out_excel_standardmodel)
allmetrics_df

,SF,overlap_abs,overlap_rel,overestimation_abs,overestimation_rel,underestimation_abs,underestimation_rel,IoU,surf_dist_median_abs,surf_dist_min,surf_dist_perc_0.5,surf_dist_perc_1,surf_dist_perc_2,surf_dist_perc_5,surf_dist_perc_25,surf_dist_perc_50,surf_dist_perc_75,surf_dist_perc_95
0,1.0,722.048713,66.567532,0.414476,0.038212,362.637306,33.432468,0.665421,1.398137,-2.344679,-2.282971,-2.235708,-2.177243,-2.068790,-1.495260,-0.714846,1.301430,2.029166
1,2.0,1042.214346,96.084427,26.718728,2.463268,42.471677,3.915573,0.937745,0.252195,-0.960629,-0.742165,-0.637453,-0.508970,-0.371317,-0.127006,0.180125,0.322939,0.489511
2,3.0,1069.162443,98.568841,187.524122,17.288332,15.523578,1.431159,0.840398,0.673841,-0.508128,-0.413066,-0.355505,-0.268002,0.091211,0.368705,0.673841,0.938052,1.313032
3,4.0,1071.995704,98.830047,305.943022,28.205676,12.690317,1.169953,0.770871,1.049636,-0.338673,-0.250390,-0.208128,-0.112225,0.140892,0.543098,1.049636,1.453675,1.900181
4,5.0,1075.792450,99.180079,386.517442,35.634039,8.893575,0.819922,0.731233,1.235760,-0.366368,-0.202936,-0.161899,0.068513,0.171375,0.648443,1.235760,1.770898,2.297121
5,6.0,1072.001158,98.830550,447.829032,41.286513,12.684870,1.169451,0.699504,1.360047,-0.369714,-0.201882,-0.135313,0.084617,0.192753,0.716955,1.360047,2.017182,2.612045
6,7.0,1078.467555,99.426703,493.529611,45.499767,6.218465,0.573296,0.683346,1.461803,-0.379760,-0.167791,-0.058656,0.116332,0.211955,0.766132,1.461803,2.154366,2.869475
7,8.0,1075.270066,99.131919,530.926130,48.947448,9.415963,0.868082,0.665550,1.541944,-0.486036,-0.142098,0.071102,0.120915,0.219702,0.837731,1.541944,2.285607,3.128325
8,9.0,1077.775879,99.362936,559.137120,51.548292,6.910147,0.637064,0.655652,1.593041,-0.415437,-0.146031,0.069743,0.129109,0.211148,0.822435,1.593041,2.355398,3.323528
